In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.colors as mcolors

# --- CSVファイルの取得 ---
script_dir = os.path.dirname(__file__)
csv_files = glob.glob(os.path.join(script_dir, "*.csv"))

if not csv_files:
    raise FileNotFoundError("同じフォルダにCSVファイルが見つかりません。")

# --- カラーバー範囲の設定 ---
color_ranges = {
    "SiO2 Etching depth": (0, 1700),
    "SiO2 Etching rate": (0, 600),
    "Mask Etching depth": (0, 1400),
    "Mask Selectivity ratio": (1, 6),
    "Vpp": (0, 1000),
    "Vdc": (-800, 0)
}

for file_path in csv_files:
    print(f"読み込み中: {file_path}")

    # CSV読み込み
    df = pd.read_csv(file_path, encoding="shift_jis", header=None)

    # 対象値名
    full_name = str(df.iloc[0,0])
    if "/" in full_name:
        target_name = full_name.split("/", 1)[1].strip()
    else:
        target_name = full_name.strip()
    print(f"対象値: {target_name}")

    # Coil, Platen, z の抽出
    coil_x = df.iloc[1,1:].astype(float).values
    platen_y = df.iloc[3:,0].astype(float).values
    z_values = df.iloc[3:,1:].astype(float).values

    # 座標リスト作成
    x_list, y_list, z_list = [], [], []
    for i, y_val in enumerate(platen_y):
        for j, x_val in enumerate(coil_x):
            x_list.append(x_val)
            y_list.append(y_val)
            z_list.append(z_values[i, j])
    x = np.array(x_list)
    y = np.array(y_list)
    z = np.array(z_list)

    # --- グリッド作成 ---
    xi = np.linspace(min(x), max(x), 200)
    yi = np.linspace(min(y), max(y), 200)
    Xi, Yi = np.meshgrid(xi, yi)

    # --- 2次元補間 ---
    Zi_linear = griddata((x, y), z, (Xi, Yi), method='linear')
    Zi_cubic = griddata((x, y), z, (Xi, Yi), method='cubic')  # データが少ないとnanになることがあります

    # --- 下限値を0に制限（Vdcは除外） ---
    if target_name != "Vdc":
        Zi_linear = np.clip(Zi_linear, 0, None)
        Zi_cubic = np.clip(Zi_cubic, 0, None)

    # --- vmin, vmax 設定 ---
    vmin, vmax = color_ranges.get(target_name, (None, None))

    # --- 3Dプロット (Linear) ---
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(Xi, Yi, Zi_linear,
                           cmap=cm.viridis,
                           vmin=vmin, vmax=vmax,
                           linewidth=0, antialiased=True,
                           alpha=0.9)
    ax.scatter(x, y, z, c='red', s=40, edgecolors="black", label="Measured")
    ax.set_xlabel("Coil [W]")
    ax.set_ylabel("Platen [W]")
    ax.set_zlabel(target_name)
    ax.set_title(f"{target_name} (3D Linear Interpolation)")
    fig.colorbar(surf, shrink=0.5, aspect=10, label=target_name)
    plt.legend()
    safe_name = re.sub(r'[\\/:*?"<>|]', '_', str(target_name))
    plt.savefig(os.path.join(script_dir, f"{safe_name}_linear_interp_3D.png"),
                dpi=300, bbox_inches="tight")
    plt.close()

    # --- 3Dプロット (Cubic) ---
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(Xi, Yi, Zi_cubic,
                           cmap=cm.viridis,
                           vmin=vmin, vmax=vmax,
                           linewidth=0, antialiased=True,
                           alpha=0.9)
    ax.scatter(x, y, z, c='red', s=40, edgecolors="black", label="Measured")
    ax.set_xlabel("Coil [W]")
    ax.set_ylabel("Platen [W]")
    ax.set_zlabel(target_name)
    ax.set_title(f"{target_name} (3D Cubic Interpolation)")
    fig.colorbar(surf, shrink=0.5, aspect=10, label=target_name)
    plt.legend()
    plt.savefig(os.path.join(script_dir, f"{safe_name}_cubic_interp_3D.png"),
                dpi=300, bbox_inches="tight")
    plt.close()

    print(f"3Dサーフェス補間のグラフを保存しました: {safe_name}")